In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/plant-seedlings-classification/test'):
    for filename in filenames:
        print(dirname)
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/dcd7ff249.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/23bc8ec4f.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/618de3d7a.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/1f5e5554e.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/aad8375e0.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/e80a259c5.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/8e6ec1ca6.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classification/test/e88bf0db9.png
/kaggle/input/plant-seedlings-classification/test
/kaggle/input/plant-seedlings-classifi

# Image Preprocessing

In [2]:
import cv2

In [3]:
import os
count=1;
all_images=[]
all_class=[];
path='/kaggle/input/plant-seedlings-classification/train/'
entries = os.listdir('/kaggle/input/plant-seedlings-classification/train/')
for entry in entries:
  for image_path in os.listdir(path+entry):
    img = cv2.imread(path+entry+'/'+image_path)
    img=cv2.resize(img,(32,32))
    GREEN_MIN = np.array([25, 52, 72],np.uint8)
    GREEN_MAX = np.array([102, 255, 255],np.uint8)
    hsv_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    frame_threshed = cv2.inRange(hsv_img, GREEN_MIN, GREEN_MAX)
    all_images.append(frame_threshed)
    all_class.append(entry)

    

 
X_train=np.array(all_images)
y_train=np.array(all_class)

In [4]:
X_train.shape

(4750, 32, 32)

# Test & Train Split

In [5]:
train_images=[]
all_class=[]
image_name=[]
path='/kaggle/input/plant-seedlings-classification/test/'


for image_path in os.listdir(path):
  img = cv2.imread(path+image_path)
  img=cv2.resize(img,(32,32))
  GREEN_MIN = np.array([25, 52, 72],np.uint8)
  GREEN_MAX = np.array([102, 255, 255],np.uint8)
  hsv_img = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  frame_threshed = cv2.inRange(hsv_img, GREEN_MIN, GREEN_MAX)
  train_images.append(frame_threshed)
  all_class.append(entry)
  image_name.append(image_path)

    

 
x_test=np.array(train_images)

# Image Classification

In [6]:

all_class=[]
path='/kaggle/input/plant-seedlings-classification/train/'
entries = os.listdir('/kaggle/input/plant-seedlings-classification/train/')
for entry in entries:
  print(entry)

Cleavers
Small-flowered Cranesbill
Scentless Mayweed
Shepherds Purse
Fat Hen
Common wheat
Charlock
Loose Silky-bent
Black-grass
Maize
Sugar beet
Common Chickweed


In [7]:
y_train=[]
train_classes=[]
path='/kaggle/input/plant-seedlings-classification/train/'
entries = os.listdir('/kaggle/input/plant-seedlings-classification/train/')
for entry in entries:
  for image_path in os.listdir(path+entry):
    train_classes.append(entry)
    
y_train=np.array(train_classes)

# Convert to Categorical

In [8]:
y_train=np.where(y_train=='Common Chickweed',5, y_train) 
y_train=np.where(y_train=='Charlock', 11, y_train) 
y_train=np.where(y_train=='Shepherds Purse', 10, y_train) 
y_train=np.where(y_train=='Black-grass', 1, y_train) 
y_train=np.where(y_train=='Cleavers', 2, y_train) 
y_train=np.where(y_train=='Scentless Mayweed', 9, y_train) 
y_train=np.where(y_train=='Common wheat', 8, y_train) 
y_train=np.where(y_train=='Fat Hen', 0, y_train) 
y_train=np.where(y_train=='Maize', 6, y_train) 

y_train=np.where(y_train=='Loose Silky-bent', 7, y_train) 
y_train=np.where(y_train=='Sugar beet', 4, y_train) 
y_train=np.where(y_train=='Small-flowered Cranesbill', 3, y_train) 

# Apply CNN

In [9]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
trainY=np_utils.to_categorical(y_train,12)

In [11]:
trainX =X_train/255
testX=x_test/255

In [12]:
trainX=trainX.reshape(trainX.shape[0],32,32,1).astype('float32') 
testX=testX.reshape(testX.shape[0],32,32,1).astype('float32') 

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 1), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=2 , padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(12, activation='softmax'))
# Compile model
epochs = 40
lrate = 0.01
decay = lrate/epochs
          
#sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] , )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)              

In [14]:

epochs=10
early_stopping = EarlyStopping(monitor='acc', patience=2, verbose=1, mode='auto')
callback_list = [early_stopping]# [stats, early_stopping]

model.fit(trainX, trainY,epochs=epochs, batch_size=32 , callbacks=callback_list)

Epoch 1/10
4750/4750 [==============================] - 2s 480us/step - loss: 1.9046 - accuracy: 0.3288
Epoch 2/10
1152/4750 [======>.......................] - ETA: 0s - loss: 1.6117 - accuracy: 0.4410

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `acc` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


4750/4750 [==============================] - 1s 137us/step - loss: 1.5295 - accuracy: 0.4589
Epoch 3/10
4750/4750 [==============================] - 1s 137us/step - loss: 1.2960 - accuracy: 0.5446
Epoch 4/10
4750/4750 [==============================] - 1s 137us/step - loss: 1.1173 - accuracy: 0.6000
Epoch 5/10
4750/4750 [==============================] - 1s 139us/step - loss: 0.9919 - accuracy: 0.6469
Epoch 6/10
4750/4750 [==============================] - 1s 160us/step - loss: 0.8974 - accuracy: 0.6838
Epoch 7/10
4750/4750 [==============================] - 1s 136us/step - loss: 0.7816 - accuracy: 0.7215
Epoch 8/10
4750/4750 [==============================] - 1s 142us/step - loss: 0.6938 - accuracy: 0.7457
Epoch 9/10
4750/4750 [==============================] - 1s 138us/step - loss: 0.6209 - accuracy: 0.7756
Epoch 10/10
4750/4750 [==============================] - 1s 138us/step - loss: 0.5511 - accuracy: 0.7939


# Model Prediction

pred_class=model.predict_classes(testX)

In [16]:
pred_class

array([ 4,  5, 11,  3, 11,  7,  3,  3,  5, 11,  5,  4,  0,  2,  0,  8,  5,
        3,  5,  7,  2,  4, 11,  4,  2,  5,  5,  2,  4,  9,  2,  7,  4,  4,
       10, 11,  5,  0,  4,  4,  0, 10, 11,  7, 10, 10,  3,  3,  2,  5,  7,
        4,  0,  5,  2,  5,  4, 11,  7,  8,  7,  2,  2,  6,  5,  7,  5,  4,
        7,  7,  5,  5, 11,  9,  2,  1, 10,  3,  5,  9, 11,  2,  3, 10,  4,
        0, 11,  7,  7,  7, 11, 10, 11, 10,  9,  2,  3,  9, 11, 10,  3,  9,
        7, 10,  7,  4, 11,  0,  0,  0,  0,  3,  7,  4,  5,  5,  3,  4,  1,
        7,  7,  9, 11,  5,  8,  2,  0,  3,  7,  7,  3,  7,  4,  0,  6,  6,
        2,  7,  1,  3,  9,  2,  9,  7,  4,  3,  8,  4, 11,  7,  5,  5,  6,
        0,  3,  7,  3,  0,  0,  7,  0,  5,  5,  0,  0,  7,  7,  9,  5, 11,
        3,  4,  5,  4,  0,  3,  6,  0, 11,  5,  6, 11,  7,  0,  4,  7,  7,
        4,  5,  9, 11,  1,  7,  7,  5,  1,  3,  5,  3, 11,  2, 10,  5,  4,
        3,  2, 11,  3,  3,  7,  0,  7,  8,  7,  3,  2,  9,  0,  5, 10, 11,
        5, 10,  4,  7,  7

In [17]:
image_name

['dcd7ff249.png',
 '23bc8ec4f.png',
 '618de3d7a.png',
 '1f5e5554e.png',
 'aad8375e0.png',
 'e80a259c5.png',
 '8e6ec1ca6.png',
 'e88bf0db9.png',
 'f8318faf1.png',
 'da9ef7858.png',
 'bf3924a57.png',
 '47f9e5d91.png',
 '19618ad6a.png',
 '54b3afd58.png',
 '05341a8a6.png',
 '0c4199daa.png',
 '0086a6340.png',
 '0c5f6c493.png',
 '08d591441.png',
 '16357b436.png',
 'f85ed9b6d.png',
 '0dba99002.png',
 '406ecb5c5.png',
 '6b9d6f8c9.png',
 'a8388a37f.png',
 'd84d37a61.png',
 '52dc7a4d6.png',
 'fa5fd1384.png',
 '0751c0bbc.png',
 'ef74dbcad.png',
 '1b6a6494d.png',
 '39b740f7e.png',
 'fb022edf9.png',
 '8455169fe.png',
 'df7cb5f87.png',
 '97844bfd5.png',
 '19b51843a.png',
 'c832e4302.png',
 '279df95f2.png',
 '60fea2ef6.png',
 '5a38ac566.png',
 'fc3e58836.png',
 '87608f7aa.png',
 '86676d627.png',
 '060f1dc84.png',
 '486e59179.png',
 '16fd2e01a.png',
 '67e185673.png',
 '03ef36742.png',
 '7d4cd07ad.png',
 'a254d71f6.png',
 '3827436f3.png',
 '0437393b1.png',
 '32c86784b.png',
 'f4e7733d4.png',
 'dce2f661

In [18]:
pred_class=np.where(pred_class=='5', 'Common Chickweed',pred_class) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


In [19]:
 
pred_class=np.where(pred_class=='5', 'Common Chickweed',pred_class) 
pred_class=np.where(pred_class== '11','Charlock', pred_class) 
pred_class=np.where(pred_class== '10','Shepherds Purse', pred_class) 
pred_class=np.where(pred_class== '1','Black-grass', pred_class) 
pred_class=np.where(pred_class== '2', 'Cleavers',pred_class) 
pred_class=np.where(pred_class== '9','Scentless Mayweed', pred_class) 
pred_class=np.where(pred_class== '8', 'Common wheat',pred_class) 
pred_class=np.where(pred_class== '0', 'Fat Hen',pred_class) 
pred_class=np.where(pred_class=='6', 'Maize', pred_class) 

pred_class=np.where(pred_class== '7', 'Loose Silky-bent',pred_class) 
pred_class=np.where(pred_class== '4', 'Sugar beet',pred_class) 
pred_class=np.where(pred_class== '3', 'Small-flowered Cranesbill',pred_class) 


In [20]:
pred_class

array(['Sugar beet', 'Common Chickweed', 'Charlock',
       'Small-flowered Cranesbill', 'Charlock', 'Loose Silky-bent',
       'Small-flowered Cranesbill', 'Small-flowered Cranesbill',
       'Common Chickweed', 'Charlock', 'Common Chickweed', 'Sugar beet',
       'Fat Hen', 'Cleavers', 'Fat Hen', 'Common wheat',
       'Common Chickweed', 'Small-flowered Cranesbill',
       'Common Chickweed', 'Loose Silky-bent', 'Cleavers', 'Sugar beet',
       'Charlock', 'Sugar beet', 'Cleavers', 'Common Chickweed',
       'Common Chickweed', 'Cleavers', 'Sugar beet', 'Scentless Mayweed',
       'Cleavers', 'Loose Silky-bent', 'Sugar beet', 'Sugar beet',
       'Shepherds Purse', 'Charlock', 'Common Chickweed', 'Fat Hen',
       'Sugar beet', 'Sugar beet', 'Fat Hen', 'Shepherds Purse',
       'Charlock', 'Loose Silky-bent', 'Shepherds Purse',
       'Shepherds Purse', 'Small-flowered Cranesbill',
       'Small-flowered Cranesbill', 'Cleavers', 'Common Chickweed',
       'Loose Silky-bent', 'Sugar 

In [21]:
cnt=0
result=[]
df_result=pd.DataFrame()
for x in pred_class:
    result.append(image_name[cnt]+","+pred_class[cnt])
    
    cnt=cnt+1


In [22]:
result=np.asarray(result)

# Export the result to Excel

In [23]:
import pandas as pd
df=pd.DataFrame(result)

In [24]:
df_result1=pd.DataFrame()
df_result1['species']=pred_class
df_result1['file']=image_name

In [25]:
result

array(['dcd7ff249.png,Sugar beet', '23bc8ec4f.png,Common Chickweed',
       '618de3d7a.png,Charlock',
       '1f5e5554e.png,Small-flowered Cranesbill',
       'aad8375e0.png,Charlock', 'e80a259c5.png,Loose Silky-bent',
       '8e6ec1ca6.png,Small-flowered Cranesbill',
       'e88bf0db9.png,Small-flowered Cranesbill',
       'f8318faf1.png,Common Chickweed', 'da9ef7858.png,Charlock',
       'bf3924a57.png,Common Chickweed', '47f9e5d91.png,Sugar beet',
       '19618ad6a.png,Fat Hen', '54b3afd58.png,Cleavers',
       '05341a8a6.png,Fat Hen', '0c4199daa.png,Common wheat',
       '0086a6340.png,Common Chickweed',
       '0c5f6c493.png,Small-flowered Cranesbill',
       '08d591441.png,Common Chickweed', '16357b436.png,Loose Silky-bent',
       'f85ed9b6d.png,Cleavers', '0dba99002.png,Sugar beet',
       '406ecb5c5.png,Charlock', '6b9d6f8c9.png,Sugar beet',
       'a8388a37f.png,Cleavers', 'd84d37a61.png,Common Chickweed',
       '52dc7a4d6.png,Common Chickweed', 'fa5fd1384.png,Cleavers',
   

In [26]:
df_result1

,species,file
0,Sugar beet,dcd7ff249.png
1,Common Chickweed,23bc8ec4f.png
2,Charlock,618de3d7a.png
3,Small-flowered Cranesbill,1f5e5554e.png
4,Charlock,aad8375e0.png
...,...,...
789,Fat Hen,785a73ab8.png
790,Fat Hen,8cf2e3e6c.png
791,Small-flowered Cranesbill,56112b92c.png
792,Scentless Mayweed,e1809cef2.png


In [27]:
df_result1.to_excel('sample_submission.xlsx',index=False)

In [29]:
df_result1

,species,file
0,Sugar beet,dcd7ff249.png
1,Common Chickweed,23bc8ec4f.png
2,Charlock,618de3d7a.png
3,Small-flowered Cranesbill,1f5e5554e.png
4,Charlock,aad8375e0.png
...,...,...
789,Fat Hen,785a73ab8.png
790,Fat Hen,8cf2e3e6c.png
791,Small-flowered Cranesbill,56112b92c.png
792,Scentless Mayweed,e1809cef2.png
